In [1]:
import numpy as np
import cvxpy as cp

# Load Data

In [24]:
O_SA = 18.96 * 1e9 # Ontario Surface Area

In [8]:
niagra_training = np.load('../Data/niagra_training.npy')
niagra_testing = np.load('../Data/niagra_testing.npy')

nbs_training = np.load('../Data/nbs_ontario_training.npy')
nbs_testing = np.load('../Data/calculated_nbs_ontario.npy')

In [31]:
# Net Basin Supply Uncertainty
nbs_uncert = {
    t : (min( [nbs_training[i::12] for i in range(12)][t-1] ),
         max( [nbs_training[i::12] for i in range(12)][t-1] )) for t in range(1, 13)
}

In [34]:
# Niagra Flow Uncertainty
nflow_uncert = {
    t : (min( [niagra_training[i::12] for i in range(12)][t-1] ),
         max( [niagra_training[i::12] for i in range(12)][t-1] )) for t in range(1, 13)
}

In [33]:
# M(t) and m(t)
surface_dict = {
        1 : (73.56, 75.26),
        2 : (73.62, 75.37),
        3 : (73.78, 75.33),
        4 : (73.97, 75.60),
        5 : (74.22, 75.73),
        6 : (74.27, 75.69),
        7 : (74.26, 75.63),
        8 : (74.15, 75.49),
        9 : (74.04, 75.24),
        10 : (73.83, 75.25),
        11 : (73.67, 75.18),
        12 : (73.57, 75.23),
    }

# Construct Simulation and Control Algorithm

In [18]:
def monthly_surface_extremes(month: int):
    month = month % 12
    return surface_dict[month]

In [19]:
def limit_flows(prev_surf, quart_month):
    if quart_month in [i for i in range(1, 13)] + [48]:
        return 11500
    else:
        if prev_surf <= 74.22: return 5950
        elif prev_surf <= 74.34: return 5950 + 1333*(prev_surf-74.22)
        elif prev_surf <= 74.54: return 6111 + 9100*(prev_surf-74.34)
        elif prev_surf <= 74.70: return 7930 + 2625*(prev_surf-74.54)
        elif prev_surf <= 75.13: return 8350 + 1000*(prev_surf-74.70)
        elif prev_surf <= 75.44: return 8780 + 3645*(prev_surf-75.13)
        elif prev_surf <= 75.70: return 9910
        elif prev_surf <= 76: return 10200
        else: return 10700

In [20]:
def g(prev_surf):
    return 555.823*(prev_surf - 0.035 - 69.474)**(1.5)

In [ ]:
def conersion_to_SA(flow_rate):
    '''
    flow rate assumed m^3/s
    '''
    return 

In [78]:
def find_x(month: int, quart_month: int, prev_flow: float, prev_surf: float):
    '''
    Construct a Robust LO 
    '''
    
    min_surf, max_surf = monthly_surface_extremes(month)
    
    nflow_i_lb = nflow_uncert[month][0] / (O_SA * 4) # empirical LB on Niagra Flow for month
    nflow_i_ub = nflow_uncert[month][1] / (O_SA * 4) # empirical UB on Niagra Flow for month
    nbs_i_lb = nbs_uncert[month][0] / 4 # empirical LB on Net Basin Supply for month
    nbs_i_ub = nbs_uncert[month][1] / (4) # empirical UB on Net Basin Supply for month

    pre_release = g(prev_surf)

    # print(pre_release)
    # print(( 657000.7 / O_SA )*pre_release)

    # Alternative Approach: minimize difference
    x = cp.Variable(1)

    total_flow = x + pre_release
    total_release = ( 657000.7 / O_SA ) * total_flow

    # Might need to build in ability for infeasible to loosen bound
    s_ub = prev_surf + nflow_i_ub + nbs_i_ub - total_release
    s_lb = prev_surf + nflow_i_lb + nbs_i_lb - total_release

    objective = cp.Minimize(x)

    surface_height = [s_ub <= max_surf, s_lb >= min_surf]
    flow_change = [cp.abs(total_flow- prev_flow) <= 700]
    max_flow = [total_flow  <= limit_flows(prev_surf, quart_month)]
    print(747*(prev_surf - 69.10)**(1.47))
    channel_capacity = [total_flow <= 747*(prev_surf - 69.10)**(1.47)]

    # One more constraint: try to incorporate next month

    constraints = surface_height + flow_change + max_flow + channel_capacity

    prob = cp.Problem(objective, constraints)
    prob.solve()

    return prob, x

In [79]:
solution, x = find_x(2, 1, 6690, 74.8)

9648.429762185495


/Users/q3/opt/anaconda3/envs/MCM-env/lib/python3.12/site-packages/cvxpy/reductions/solvers/solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


In [80]:
solution.status

'optimal'

In [82]:
total_flow = x.value[0] + g(74.8)

In [85]:
s_new = 74.8 + niagra_testing[0]/O_SA + nbs_testing[0] - (total_flow/O_SA)*2.6784*1e6

In [86]:
s_new

75.06786444083811

In [ ]:
# Environment
'''
Time Horizon: January 2012-December 2020
'''

# 


In [ ]:
# Need to properly subtract flow: with SA